# import package

In [121]:
import requests, time, json
import pandas as pd
from bs4 import BeautifulSoup
from requests.api import request

# API collection

<b>＊＊＊股市＊＊＊</b>
<br>
1. 台灣：
    <br>-台灣加權
    <br>-台指期
    <br>
    
2. 美國：
    <br>-費城半導體
    <br>-納斯達克
    <br>-S&P500
    <br>-美元指數
3. 日本：
    <br>-日經指數
4. 中國：
    <br>-上海A股
    <br>-上海B股
    <br>-深圳A股
    <br>-深圳B股
<br>
<b>＊＊＊美元匯率＊＊＊</b>
<br>
1. 美元指數
2. 台灣
3. 日本
4. 歐元
5. 澳幣
6. 人民幣
<br>
<b>＊＊＊期貨＊＊＊</b>
<br>
1. 黃金價格
2. 

In [ ]:
# 
"""
指數url
https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=D
&symbol=GI:NKY:INDEX
&from=1638165061
&to=1603259401

指數種類url
https://ws.api.cnyes.com/ws/api/v2/universal/quote?type=LMMR
"""

'''
日：https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=D&symbol=GI:NKY:INDEX&from=1638165061&to=1603259401
每週：https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=W&symbol=GI:NKY:INDEX&from=1638165255&to=1456811595
'''

# define function

In [87]:
# exchange datetime.
def timestamp_to_localtime(time_stamp):
    time_trans = time.localtime(time_stamp)
    local_time = time.strftime("%Y-%m-%d %H:%M:%S", time_trans)
    return local_time

def localtime_to_timestamp(datetime, style="%Y-%m-%d %H:%M:%S"):
    time_trans = time.strptime(datetime, style)
    stamp_time = time.mktime(time_trans)
    return int(stamp_time)
# example
print(timestamp_to_localtime(1638196277))
print(localtime_to_timestamp("2021-11-27 00:00:00", "%Y-%m-%d %H:%M:%S"))

2021-11-29 22:31:17
1637942400


In [122]:
'''
resolution：日(D)、週(W)、月(M)均價。
symbol：指數。
date_from - date_to：從幾號起往“回”推到幾號。(Ex. 2021-11-29 -> 2020-11-29)
'''
def crawler(resolution, symbol, date_from, date_to): #
    date_from = str(localtime_to_timestamp(date_from))
    date_to = str(localtime_to_timestamp(date_to))
    url = 'https://ws.api.cnyes.com/ws/api/v1/charting/history?resolution=' + resolution + '&symbol=' + symbol + '&from=' + date_from + '&to=' + date_to
    res_url = requests.get(url)
    data_date = [timestamp_to_localtime(time) for time in json.loads(res_url.text)['data']['t']]
    data_open = json.loads(res_url.text)['data']['o']
    data_low = json.loads(res_url.text)['data']['l']
    data_high = json.loads(res_url.text)['data']['h']
    data_close = json.loads(res_url.text)['data']['c']
    data = {'data_date': data_date, 
            'data_open': data_open, 
            'data_high': data_high, 
            'data_low': data_low, 
            'data_close': data_close
           }
    return pd.DataFrame(data)


# 爬蟲

## 指數

### 指數搜集

In [159]:
# url = https://ws.api.cnyes.com/ws/api/v3/universal/quote?type=IDXMAJOR&column=B&page=1&limit=10
url_index = 'https://ws.api.cnyes.com/ws/api/v3/universal/quote?type=IDXMAJOR&column=B' #&column=C&page=1&limit=0
request_index = requests.get(url_index)
index_list = dict()
for data in json.loads(request_index.text)['data']['items']:
    index_list[data['0']] = data['200009']
print(index_list)

{'GI:NKY:INDEX': '日經225指數', 'GI:KOSPI:INDEX': '南韓綜合指數', 'TWS:TSE01:INDEX': '台灣加權指數', 'TWS:OTC01:INDEX': '台灣櫃檯指數', 'GI:HSI:INDEX': '恆生指數', 'GI:HSCE:INDEX': '恆生國企', 'GI:SSEC:INDEX': '上證指數', 'GI:SSEA:INDEX': '上證A股', 'GI:SSE50:INDEX': '上證50', 'GI:CSI300:INDEX': '滬深300', 'GI:SZ100:INDEX': '深證100', 'GI:FTSE:INDEX': '英國FTSE100', 'GI:CAC:INDEX': '法國CAC40', 'GI:NQDELC:INDEX': 'NQ德國指數', 'GI:DJI:INDEX': '道瓊指數', 'GI:INX:INDEX': 'S&P 500', 'GI:IXIC:INDEX': 'NASDAQ', 'GI:SOX:INDEX': '費城半導體', 'GF:CL:FUTURES': '近月紐約輕原油', 'GI:XAUUSDOZ:INDEX': '現貨黃金', 'GI:DXY:INDEX': '美元指數', 'GI:FTXIN9:INDEX': '富時中國A50', 'GI:FTXIN150:INDEX': '富時中國A150', 'GI:HXR:INDEX': 'NQ全球資安指數', 'GI:NQROBOT:INDEX': 'NQROBO指數', 'TWS:LV30:INDEX': '臺灣股利精選30', 'EOD:TWESGHY30:EOD': 'MSCI臺灣ESG永續高股息精選30', 'TWS:IX0139:INDEX': '臺灣5G+通訊指數'}


### 指數爬蟲

In [161]:
for i in list(index_list.keys()):
    INDEX_df = FX_crawler('D', i, '2021-11-29 00:00:00', '2020-11-29 00:00:00')
    print(index_list[i])
    print(INDEX_df)

日經225指數
                 FX_date   FX_open   FX_high    FX_low  FX_close
0    2021-11-26 08:00:00  29324.47  29332.99  28605.61  28751.62
1    2021-11-25 08:00:00  29469.65  29570.42  29444.45  29499.28
2    2021-11-24 08:00:00  29663.45  29758.05  29212.93  29302.66
3    2021-11-22 08:00:00  29618.55  29806.49  29542.29  29774.11
4    2021-11-19 08:00:00  29641.05  29768.54  29589.19  29745.87
..                   ...       ...       ...       ...       ...
237  2020-12-04 08:00:00  26697.26  26799.83  26646.08  26751.24
238  2020-12-03 08:00:00  26740.30  26868.09  26719.23  26809.37
239  2020-12-02 08:00:00  26884.99  26889.90  26695.28  26800.98
240  2020-12-01 08:00:00  26624.20  26852.16  26618.39  26787.54
241  2020-11-30 08:00:00  26830.10  26834.20  26405.83  26433.62

[242 rows x 5 columns]
南韓綜合指數
                 FX_date  FX_open  FX_high   FX_low  FX_close
0    2021-11-26 08:00:00  2973.04  2985.77  2930.31   2936.44
1    2021-11-25 08:00:00  2996.74  2996.74  2973.24   298

深證100
                 FX_date    FX_open    FX_high     FX_low   FX_close
0    2021-11-26 08:00:00  6874.6290  6906.2462  6841.0670  6853.7842
1    2021-11-25 08:00:00  6916.3432  6926.4966  6873.8276  6886.3912
2    2021-11-24 08:00:00  6927.4586  6946.9525  6904.8293  6919.5456
3    2021-11-23 08:00:00  6949.8810  6952.0181  6907.1881  6925.1396
4    2021-11-22 08:00:00  6891.2062  6967.2437  6884.2856  6967.2437
..                   ...        ...        ...        ...        ...
237  2020-12-04 08:00:00  6537.1080  6594.0564  6525.0927  6583.3276
238  2020-12-03 08:00:00  6554.3405  6575.4101  6518.9689  6557.7686
239  2020-12-02 08:00:00  6553.1905  6571.9273  6504.5546  6555.2668
240  2020-12-01 08:00:00  6399.2083  6542.1913  6398.9054  6539.0389
241  2020-11-30 08:00:00  6430.0210  6477.8495  6396.3416  6405.7788

[242 rows x 5 columns]
英國FTSE100
                 FX_date  FX_open  FX_high   FX_low  FX_close
0    2021-11-26 08:00:00  7310.37  7310.37  7042.12   7044.03
1    202

富時中國A150
                 FX_date   FX_open   FX_high    FX_low  FX_close
0    2021-11-26 08:00:00  21207.98  21264.98  21102.36  21149.80
1    2021-11-25 08:00:00  21281.11  21285.47  21186.58  21207.98
2    2021-11-24 08:00:00  21366.40  21417.18  21257.78  21281.11
3    2021-11-23 08:00:00  21393.03  21449.77  21309.14  21366.40
4    2021-11-22 08:00:00  21153.61  21402.26  21149.20  21393.03
..                   ...       ...       ...       ...       ...
236  2020-12-04 08:00:00  20152.54  20187.59  19986.93  20156.65
237  2020-12-03 08:00:00  20252.27  20252.27  20101.65  20152.54
238  2020-12-02 08:00:00  20220.12  20325.50  20141.37  20252.27
239  2020-12-01 08:00:00  19782.40  20234.46  19742.91  20220.12
240  2020-11-30 08:00:00  19738.34  20033.44  19727.97  19782.40

[241 rows x 5 columns]
NQ全球資安指數
                 FX_date   FX_open   FX_high    FX_low  FX_close
0    2021-11-26 08:00:00  663.0333  663.0866  651.5809  653.9824
1    2021-11-24 08:00:00  659.9761  667.0645  65

## 匯率

### define 匯率 function

### 取資料

In [78]:
# 匯率資料：對應名稱。
GI_url = 'https://ws.api.cnyes.com/ws/api/v2/universal/quote?type=LCWR&page=0&limit=100&column=A_FORMAT'
GI_url_res = requests.get(GI_url)
data = json.loads(GI_url_res.text)['data']['items']
# 
forex_name = dict()
for data in json.loads(GI_url_res.text)['data']['items']:
    if 'FX' in data['0']:
        forex_name[data['0']] = data['200009']

{'FX:USDTWD:FOREX': '美元/台幣', 'FX:USDCNY:FOREX': '美元/人民幣', 'FX:EURUSD:FOREX': '歐元/美元', 'FX:USDJPY:FOREX': '美元/日幣', 'FX:GBPUSD:FOREX': '英鎊/美元', 'FX:AUDUSD:FOREX': '澳幣/美元'}


In [137]:
for i in list(forex_name.keys()):
    FX_df = FX_crawler('D', i, '2021-11-29 00:00:00', '2020-11-29 00:00:00')
    print(i)
    print(FX_df)

FX:USDTWD:FOREX
                 FX_date  FX_open  FX_high  FX_low  FX_close
0    2021-11-26 08:00:00    27.85   27.870  27.793    27.836
1    2021-11-25 08:00:00    27.81   27.810  27.781    27.801
2    2021-11-24 08:00:00    27.85   27.850  27.782    27.793
3    2021-11-23 08:00:00    27.84   27.850  27.788    27.809
4    2021-11-22 08:00:00    27.84   27.840  27.770    27.783
..                   ...      ...      ...     ...       ...
241  2020-12-04 08:00:00    28.43   28.521  28.250    28.521
242  2020-12-03 08:00:00    28.77   28.770  28.401    28.668
243  2020-12-02 08:00:00    28.81   28.810  28.500    28.766
244  2020-12-01 08:00:00    28.84   28.840  28.515    28.837
245  2020-11-30 08:00:00    28.81   28.824  28.500    28.824

[246 rows x 5 columns]
FX:USDCNY:FOREX
                 FX_date  FX_open  FX_high  FX_low  FX_close
0    2021-11-26 08:00:00   6.3884   6.3936  6.3845    6.3924
1    2021-11-25 08:00:00   6.3909   6.3910  6.3817    6.3859
2    2021-11-24 08:00:00   6.